In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [ ]:
!huggingface-cli

usage: huggingface-cli <command> [<args>]

positional arguments:
  {download,upload,repo-files,env,login,whoami,logout,auth,repo,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache,tag,version,upload-large-folder}
                        huggingface-cli command helpers
    download            Download files from the Hub
    upload              Upload a file or a folder to a repo on the Hub
    repo-files          Manage files in a repo on the Hub
    env                 Print information about the environment.
    login               Log in using a token from huggingface.co/settings/tokens
    whoami              Find out which huggingface.co account you are logged in as.
    logout              Log out
    auth                Other authentication related commands
    repo                {create} Commands to interact with your huggingface.co repos.
    lfs-enable-largefiles
                        Configure your repository to enable upload of files > 5GB.
    scan-cache 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_response(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(**inputs,
                                max_length=max_length,
                                temperature=0.7,
                                repetition_penalty=1.2,
                                top_k=50,
                                top_p=0.95)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def interview_approach(question):
    prompt = (
        f"You are an expert interviewer. Given the question: '{question}', "
        "ask 3 clarifying questions before answering in detail."
    )
    return generate_response(prompt, max_length=150)

def cot_approach(question):
    prompt = f"Solve the following problem step by step: {question}"
    return generate_response(prompt, max_length=150)

def tot_approach(question):
    prompt = f"Break down the problem into multiple possible approaches and explore each before concluding: {question}"
    return generate_response(prompt, max_length=150)

def zero_shot_prompt(question):
    return generate_response(question, max_length=100)

def few_shot_prompt(question, examples):
    example_text = "\n".join([f"Example {i+1}: {ex}" for i, ex in enumerate(examples)])
    prompt = f"{example_text}\nNow, answer the following: {question}"
    return generate_response(prompt, max_length=150)

test_question = "How is AI transforming healthcare?"
examples = [
    "AI helps in diagnosing diseases faster.",
    "AI enhances personalized treatment plans.",
    "AI improves operational efficiency in hospitals."
]

interview_result = interview_approach(test_question)
cot_result = cot_approach(test_question)
tot_result = tot_approach(test_question)
zero_shot_result = zero_shot_prompt(test_question)
few_shot_result = few_shot_prompt(test_question, examples)

print("\nInterview Approach ")
print(interview_result)
print("\nChain of Thought (CoT) :")
print(cot_result)
print("\nTree of Thought (ToT) :")
print(tot_result)
print("\nZero Shot :")
print(zero_shot_result)
print("\nFew Shot :")
print(few_shot_result)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Interview Approach 
You are an expert interviewer. Given the question: 'How is AI transforming healthcare?', ask 3 clarifying questions before answering in detail.
The first one will be about how you can help improve your skills and understanding of what it means to work with people who have a disability, or those that suffer from mental health issues such as depression, anxiety etc. The second two answers may also include some basic information on why they need assistance (e-mailing them) but not all responses should contain any specific advice for working out if there's anything wrong - this could mean asking whether someone has been diagnosed by their doctor at least once since last year; these might even suggest something like "I'm sure I've had my fair share" which would

Chain of Thought (CoT) :
Solve the following problem step by step: How is AI transforming healthcare?
The answer to this question depends on how much of a change in technology we are seeing. The most obvious exa